In [1]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image, ImageFile
import numpy as np
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [12]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# other imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pickle

In [3]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/'

In [4]:
from sklearn.model_selection import StratifiedKFold, KFold
import xgboost as xgb
params = {
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 5,
}

In [5]:
import xgbfir


# sub_preds = np.zeros(test_.shape[0])

feature_importance_df = pd.DataFrame()

In [6]:
pth = train_dir+'train_features_'+str(0)+'.npy'
lbls_pth = train_dir+'y_train_'+str(0)+'.npy'
train = np.load(pth)
print('data read in')

data read in


In [7]:
y_train = np.load(lbls_pth)
print('labels read in')

labels read in


In [8]:
trn_x, val_x, trn_y, val_y = train_test_split(train, y_train, test_size = 0.2)
print('data split')

data split


In [9]:
clf = xgb.XGBClassifier(objective="multi:softprob", verbosity = 2, random_state=42)
clf.fit(trn_x,trn_y)

[12:15:53] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:54] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:54] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:54] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:55] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:56] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:56] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:57] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[12:15:57] INFO: src/tree/updater_prune.cc:74: tree prun

KeyboardInterrupt: 

In [15]:
for cycle in range(2):
    pth = train_dir+'train_features_'+str(cycle)+'.npy'
    lbls_pth = train_dir+'y_train_'+str(cycle)+'.npy'
    train = np.load(pth)
    print('data read in')
    y_train = np.load(lbls_pth)
    print('labels read in')
    trn_x, val_x, trn_y, val_y = train_test_split(train, y_train, test_size = 0.2)
    print('data split')
    dtrain = xgb.DMatrix(trn_x, trn_y)
    val_train = xgb.DMatrix(val_x, val_y)
    oof_preds = np.zeros(len(val_y))
    gc.collect()
    if cycle == 0:
#         clf = xgb.train(params, dtrain)
        clf = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
        clf.fit(trn_x,trn_y)
        # save model to file
        clf.save_model('xgb_190903.bin')
    else:
        clf = xgb.XGBClassifier()
        clf.fit(trn_x,trn_y,'xgb_190903.bin')
        gc.collect()
    
    oof_preds = clf.predict(val_x)
    
    xgbfir.saveXgbFI(clf, feature_names=list(range(trn_x.shape[0])), OutputXlsxFile='ieee_xgbfir_%sFold.xlsx'%str(n_fold+1), MaxInteractionDepth=3, MaxHistograms=15)
    gc.collect()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].index
    fold_importance_df["fscore"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].values
    fold_importance_df["fold"] = cycle + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    gc.collect()
#     print(cohen_kappa_score(y_test, y_pred))
    print('\nFold %2d cohens kappa %.6f & std %.6f' %(cycle + 1, cohen_kappa_score(val_y, oof_preds[val_idx]), np.std([oof_preds])))
    gc.collect()
# print('\nCV cohens kappa score score %.6f & std %.6f' % (cohen_kappa_score(y_train, oof_preds), np.std((oof_preds))))


data read in
labels read in
data split


NameError: name 'n_fold' is not defined

In [13]:
cohen_kappa_score(val_y,oof_preds)

0.4081900318480165